In [7]:
import torch
import numpy as np
import transforms_data as td
from PIL import Image
import glob
from torch import cuda


CATEGORIES = ["Falco", "Jesse", "Konrad", "Nils", "Johannes"]
CATEGORIES.sort()
AUTHORIZED = ["Falco","Konrad"]
RESIZE_Y = 150
RESIZE_X = 100
DATA_TEST_FOLDER = "../test/*png"

model = torch.load('./class_sample/model.pt')

In [8]:
image_array = []
files = glob.glob (DATA_TEST_FOLDER)
files.sort()
# declare function of transformation
preprocess = td.transforms_valid_and_test((RESIZE_Y, RESIZE_X),[0.485, 0.456, 0.406], [0.229, 0.224, 0.225])

for f in files:
    image = Image.open(f)
    image_transformed = preprocess(image)
    image_transformed = image_transformed.reshape(-1, RESIZE_Y, RESIZE_X, 1)
    image_transformed = image_transformed.permute(3, 0, 1, 2)
    if cuda.is_available():
        image_array.append(image_transformed.type('torch.cuda.FloatTensor'))
    else:
        image_array.append(image_transformed.type('torch.FloatTensor'))

In [9]:
all_classes = []
summ_pred = np.empty(1)
for i in image_array:
	with torch.no_grad():
		pred = model(i)
		pred = torch.softmax(pred, 1)
		pred = pred.cpu().numpy()
		summ_pred = summ_pred + pred

	classes = np.argmax(pred, 1)
	all_classes.append(classes[0])

	pred = np.append(pred, classes)
	pred = np.append(pred, CATEGORIES[classes[0]])	
	print(pred, "\n")
print(all_classes)
print(summ_pred)

['0.9667887091636658' '0.0005871779285371304' '0.02024783194065094'
 '0.012376310303807259' '0.0' 'Falco'] 

['0.9290536046028137' '0.003952649887651205' '0.02345476858317852'
 '0.04353899508714676' '0.0' 'Falco'] 

['0.9492008686065674' '0.02982499450445175' '0.011885423213243484'
 '0.009088710881769657' '0.0' 'Falco'] 

['0.9383549690246582' '0.0022658249363303185' '0.03458288311958313'
 '0.024796441197395325' '0.0' 'Falco'] 

['0.8705261945724487' '0.09643664956092834' '0.029231686145067215'
 '0.0038054778706282377' '0.0' 'Falco'] 

['0.27524513006210327' '0.20828931033611298' '0.48518282175064087'
 '0.03128273785114288' '2.0' 'Johannes'] 

['0.7961697578430176' '0.17696717381477356' '0.01918684132397175'
 '0.007676254026591778' '0.0' 'Falco'] 

['0.8900232315063477' '0.042689815163612366' '0.049403268843889236'
 '0.017883557826280594' '0.0' 'Falco'] 

[0, 0, 0, 0, 0, 2, 0, 0]
[[6.61536247 0.5610136  0.67317552 0.15044849]]


In [10]:
# Hier besser die Warscheinlichkeit über alle Bilder ermitteln und darüber prüfen.
# Beispiel: Bei 5 Bilder muss die aufsummierte Wahrscheinlichkeit für eine Person >4 sein!!

NUMBER_AUTHORIZED = int(.7*len(image_array))
authentification_dict = {CATEGORIES[i]:all_classes.count(i) for i in all_classes}
print(authentification_dict) 

for a in authentification_dict:
    if a in AUTHORIZED and summ_pred[0][CATEGORIES.index(a)]>= NUMBER_AUTHORIZED:
        print("Access granted! Welcome "  + a + "!")
        break
    else:
        print("Access denied")
        break


{'Falco': 7, 'Johannes': 1}
Access granted! Welcome Falco!
